In [25]:
import tensorflow as tf
import cv2 as cv

In [26]:
def get_model(base_trainable=False):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights="imagenet", input_shape=(96, 96, 3))
    base_model.trainable = base_trainable
    
    inputs = tf.keras.Input(shape=(96, 96, 3))
    x = tf.keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(30)(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [29]:
model = get_model()

In [3]:
base_model = tf.keras.applications.ResNet50(include_top=False, weights="imagenet", input_shape=(96, 96, 3))
base_model.trainable = False

2021-08-10 23:08:55.178966: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-10 23:08:55.179001: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-10 23:08:55.179025: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ali-Latitude-E5470): /proc/driver/nvidia/version does not exist
2021-08-10 23:08:55.179253: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
inputs = tf.keras.Input(shape=(96, 96, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(30)(x)
model = tf.keras.Model(inputs, outputs)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
resnet50 (Functional)        (None, 3, 3, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 30)                61470     
Total params: 23,649,182
Trainable params: 61,470
Non-trainable params: 23,587,712
_________________________________________________________________


In [10]:
import pandas as pd

In [11]:
!ls ../data

IdLookupTable.csv  SampleSubmission.csv  test.csv  training.csv


In [12]:
train_df = pd.read_csv("../data/training.csv")

In [14]:
train_df = train_df.fillna(0)

In [15]:
no_rows, no_columns = train_df.shape

In [16]:
y_s = []
for i in range(no_rows):
    y_s.append(train_df.iloc[i][:-1].values.astype(float))

In [17]:
import numpy as np

def get_image_from_string(image_pixels_string):
    
    image_pixels_list = [int(point) for point in image_pixels_string.split(" ")]
    image_side = int(np.sqrt(len(image_pixels_list)))
    image = np.reshape(image_pixels_list, (image_side, image_side))
    image = np.stack([image, image, image], axis=2)
    
    return image

In [18]:
x_s = []
for i in range(no_rows):
    x_s.append(get_image_from_string(train_df.iloc[i]["Image"]).astype(float))

In [19]:
x_batch = np.empty((7049,96,96, 3))
for i in range(len(x_s)):
    x_batch[i,:,:] = x_s[i]

In [20]:
y_batch = np.empty((7049, 30))
for i in range(len(y_s)):
    y_batch[i,:] = y_s[i]

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='mean_squared_error')

history = model.fit(x_batch, y_batch, epochs=1)

134/221 [=================>............] - ETA: 39s - loss: 246.9299

KeyboardInterrupt: 